In [24]:
# імпорт бібліотек
import cv2
import dlib
from ultralytics import YOLO
import os
from tkinter import filedialog

## Допоміжні функції 

In [25]:
def detected_object(conf, box, class_name):
    """
    Створює словник із параметрами виявленого об'єкта.

    Параметри:
    - `conf`: Достовірність виявлення об'єкта.
    - `box`: Координати прямокутника, що обмежує об'єкт (x1, y1, x2, y2).
    - `class_name`: Назва класу об'єкта.

    Повертає:
    - `Словник із параметрами виявленого об'єкта`, який містить 'conf' (достовірність), 'box' (координати прямокутника), та 'class' (назва класу).
    """
    x1, y1, x2, y2 = map(int, box)
    return {'conf': conf, 'box': [x1, y1, x2, y2], 'class': class_name}

### <span style="color:lightblue">Аналіз тестового відео та об'єктів на ньому</span>

In [26]:
# Загрузка модели YOLO
model = YOLO('yolov5su.pt')

# Открытие видеофайла
video = cv2.VideoCapture('./test_videos/cars2.mp4')

# Инициализация списка для хранения обнаруженных объектов и чтение первого кадра
detected_objects = []
success, frame = video.read()

# Получение детекций от модели YOLO
detections = model(frame)

# Обработка детекций и сохранение информации об объектах
for detection in detections:
    for box in detection.boxes:
        cls = int(box.cls[0])
        class_name = model.names[cls]
        if cls == 2:
            conf = box.conf[0]
            detected_objects.append(detected_object(conf, box.xyxy[0], class_name))

# Выбор объектов с наивысшей и наименьшей уверенностью            
high_conf_object = max(detected_objects, key=lambda obj: obj['conf'])
low_conf_object = min(detected_objects, key=lambda obj: obj['conf'])

video.release()



0: 384x640 3 persons, 8 cars, 1 truck, 47.5ms
Speed: 1.8ms preprocess, 47.5ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)


### <span style="color:lightblue">Вивід нового відео з відображенням двох найбільш та найменш опознаних об'єктів</span>

In [27]:
# Инициализация трекеров
tracker_high = dlib.correlation_tracker()
tracker_low = dlib.correlation_tracker()

# Получение начальных координат боксов для трекинга
high_box = high_conf_object['box']
low_box = low_conf_object['box']

# Старт трекинга
tracker_high.start_track(frame, dlib.rectangle(*high_box))
tracker_low.start_track(frame, dlib.rectangle(*low_box))

# новый объект видео для второго цикла обработки кадров
video = cv2.VideoCapture('./test_videos/cars2.mp4')
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('./result_videos/output.mp4', fourcc, fps, (width, height))

In [28]:
# success и frame из второго цикла
while success:
    tracker_high.update(frame)
    tracker_low.update(frame)
    pos_high = tracker_high.get_position()
    pos_low = tracker_low.get_position()

    class_high = high_conf_object['class']
    class_low = low_conf_object['class']

    cv2.rectangle(frame, (int(pos_high.left()), int(pos_high.top())), (int(pos_high.right()), int(pos_high.bottom())), (0, 255, 0), 3)
    cv2.rectangle(frame, (int(pos_low.left()), int(pos_low.top())), (int(pos_low.right()), int(pos_low.bottom())), (0, 238, 255), 3)

    # текст к квадратам
    cv2.putText(frame, f'{class_high} {high_conf_object["conf"]:.2f}', (int(pos_high.left()), int(pos_high.top()) - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(frame, f'{class_low} {low_conf_object["conf"]:.2f}', (int(pos_low.left()), int(pos_low.top()) - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 238, 255), 2)

    out.write(frame)

    success, frame = video.read()

video.release()
out.release()